In [10]:
import gensim
from gensim.models import Word2Vec
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import codecs, json 
import nltk
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
with open("/content/drive/MyDrive/Colab Notebooks/apt/Dataset/dataframes/df_preprocessed2.csv", 'r') as f:
  df_preprocessed = pd.read_csv(filepath_or_buffer=f, sep=',')


In [13]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [14]:
#path="/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [15]:
def get_vector_average(words):
  vectors = 0
  numOfExceptableWords = 0  #ukupni broj rijeci za koje cemo izgenerirati vektore
  for w in words:
    try:                    #ako neka rijec ne postoji u vokabularu baca se iznimka
      vec = model.wv[w]
      vectors += vec
      numOfExceptableWords +=1
    except:
      continue
  
  listV=list()
  if(numOfExceptableWords==0): 
    for i in range(300):
      listV.append(str(0.0))
  else: 
    vecM=vectors/numOfExceptableWords
    for v in vecM:
      listV.append(str(v))
    
  return listV

In [16]:
d= {'id': [], 'query_vector': [], 'product_vector': [], 'median_relevance': [], 'relevance_variance': []}
df_pp_vectors = pd.DataFrame(data=d)

query_average = []
products_average = []
ids = []
medians = []
variances = []

 
for i, item in df_preprocessed.iterrows():
  ids.append(item["id"])
  medians.append(item["median_relevance"])
  variances.append(item["relevance_variance"])
  query_words = item["query"].split()
  product_words = item["product"].split()
  
  query_average.append(get_vector_average(item["query"].split()))
  products_average.append(get_vector_average(item["product"].split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [17]:
df_pp_vectors['id']=ids
df_pp_vectors['query_vector']=query_average
df_pp_vectors['product_vector']=products_average
df_pp_vectors['median_relevance']=medians
df_pp_vectors['relevance_variance']=variances

In [18]:
df = {}
df["entity"] = list()
for i, o in df_pp_vectors.iterrows(): 
  df["entity"].append({
         "id": o.id,
         "query_vector": o.query_vector,
         "product_vector": o.product_vector,
         "median_relevance": o.median_relevance,
         "relevance_variance": o.relevance_variance
  })
 
with open('/content/drive/MyDrive/Colab Notebooks/apt/df_pp_word2vec.json', 'w') as outfile:
  json.dump(list(df["entity"]),outfile,indent=3)